In [72]:
# Instructions
# Consider that in a room with MxN spaces, there are P reactive cleaning robots. Each of the robot cleaners behaves as follows:

# If the cell you are in is dirty, then vacuum for 10 seconds.
# If the cell is clean, the robot chooses a random direction to move (one of the 8 neighboring cells that is without another robot) and chooses the move action (if it cannot move there, it will stay in the same cell). This move lasts 2 seconds.
# If several robots coincide in any of the cells, one remains in that cell and the others have to move to some other cell. Who stays or who has to move is chosen randomly.
# At the beginning of the simulation, the positions of the robots are chosen at random, and in the same way the positions are clean or dirty randomly.

In [73]:
import agentpy as ap
import numpy as np
import matplotlib.pyplot as plt
import IPython, copy

In [74]:
class CleaningRobot(ap.Agent):
    def setup(self):
      self.moves = 0
      self.state = -1 # -1: searching, 0: cleaned finished, 1,2,3,4,5: cleaning
    
    def next(self):
      pos = self.model.room.positions[self]

      # If the cell is clean keep searching
      if self.model.room_state[pos] == 0:
        self.state = -1
        self.move()
      else:
        if self.state == -1:
          self.state = 5
        elif self.state > 0:
          self.state -= 1
        else:
          self.model.room_state[pos] = 0
          self.state -= -1


    def move(self):
      # Move to a random position with a state of 1
      posM = self.model.random.randint(0, self.p.M - 1)
      posN = self.model.random.randint(0, self.p.N - 1)
      pos = (posM, posN)

      # If the position is 1 and not occupied, move there
      while self.model.room_state[pos] != 1:
        posM = self.model.random.randint(0, self.p.M - 1)
        posN = self.model.random.randint(0, self.p.N - 1)
        pos = (posM, posN)
      self.model.room.positions[self] = pos

In [75]:
class CleaningRobotsModel(ap.Model):

  def setup(self):
    # Initialize robots
    self.robots = ap.AgentList(self, self.p.robots, CleaningRobot)

    # Initialize environment
    self.room = ap.Grid(self, [self.p.M, self.p.N], track_empty=True)
    self.room.add_agents(self.robots, random=True, empty=True)
    self.room_state = np.zeros([self.p.M, self.p.N])

    # Add dirty cells
    # self.room_state[self.random.randint(0, self.p.M, self.p.dirty_cells), self.random.randint(0, self.p.N, self.p.dirty_cells)] = 1

    n_dirty_cells = int(self.p.M * self.p.M * self.p.dirty_cells)

    for _ in range(n_dirty_cells):
      row = self.random.randint(0, self.p.M - 1)
      col = self.random.randint(0, self.p.N - 1)
      if self.room_state[row, col] == 0:
        self.room_state[row, col] = 1

  def step(self):
    self.robots.next()

    # Resolve conflicts
    
    if np.sum(self.room_state.flatten()) == 0:
      self.stop()

  def update(self):
    n_moves = np.sum(np.array(self.robots.moves))
    self.record('Moves', n_moves)

    n_dirty = np.sum(self.room_state.flatten())
    self.record('Dirty cells', n_dirty)

    if n_dirty == 0:
      self.stop()
  
  def end(self):
    n_moves = np.sum(np.array(self.robots.moves))
    self.report('Moves', n_moves)
    self.report('Time', 2 * self.t)

In [76]:
def animation_plot(model, ax):
  grid = copy.deepcopy(model.room_state)

  for robot in model.robots:
    grid[model.room.positions[robot]] = 2
  
  color_dict = {
    0: '#E5E5E5',
    1: '#D62C2C',
    2: '#207567',
  }
  ap.gridplot(grid, ax=ax, color_dict=color_dict, convert=True)
  ax.set_title(f'Cleaning robots\n'
               f'Time-step: {model.t * 2}, Cels left: '
               f'{np.sum(model.room_state).flatten()}')

In [77]:
parameters = {
  'M': 20,            # Number of rows
  'N': 10,            # Number of columns
  'robots': 5,        # Number of robots
  'dirty_cells': 0.2, # Percentage of dirty cells
  'steps': 100,       # Number of steps
}

In [80]:
fig, ax = plt.subplots()
model = CleaningRobotsModel(parameters)
animation = ap.animate(model, fig, ax, animation_plot)
IPython.display.HTML(animation.to_jshtml(fps=15))